In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
#         self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

if __name__ == '__main__':
    import numpy as np 
    
    use_cuda = torch.cuda.is_available()

    torch.manual_seed(0)

    device = torch.device("cuda" if use_cuda else "cpu")

    model = Encoder().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    model.train()
    for itr in range(10000):
        x = np.random.uniform(size=(100,2))
        y = np.random.uniform(size=(100,2))
        targ = (np.linalg.norm(x-y, axis=1) <= 0.4).reshape(-1, 1).astype(np.float32)
        attn = model(torch.Tensor(np.concatenate((x, y), axis=1), device=device))
#         attn = model(torch.Tensor(np.linalg.norm(x-y, axis=1).reshape(-1, 1), device=device))
#         e_x = model(torch.Tensor(x, device=device))
#         e_y = model(torch.Tensor(y, device=device))
        optimizer.zero_grad()
        loss = torch.mean(
            torch.abs(attn - torch.Tensor(targ, device=device)),
        )
        if itr % 1000 == 0:
            print(loss)
        loss.backward()
        optimizer.step()
    print(loss)
    x = np.random.uniform(size=(10,2))
    y = np.random.uniform(size=(10,2))
    targ = (np.linalg.norm(x-y, axis=1) <= 0.4).reshape(-1, 1).astype(np.float32)
    attn = model(torch.Tensor(np.concatenate((x, y), axis=1), device=device))
    print(targ)
    print(attn)

tensor(0.4452, grad_fn=<MeanBackward1>)
tensor(0.1108, grad_fn=<MeanBackward1>)
tensor(0.0865, grad_fn=<MeanBackward1>)
tensor(0.0868, grad_fn=<MeanBackward1>)
tensor(0.0661, grad_fn=<MeanBackward1>)
tensor(0.0885, grad_fn=<MeanBackward1>)
tensor(0.0507, grad_fn=<MeanBackward1>)
tensor(0.0778, grad_fn=<MeanBackward1>)
tensor(0.0903, grad_fn=<MeanBackward1>)
tensor(0.0467, grad_fn=<MeanBackward1>)
tensor(0.0767, grad_fn=<MeanBackward1>)
[[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]]
tensor([[-0.0656],
        [ 0.9974],
        [ 0.9974],
        [ 0.9974],
        [ 0.5229],
        [ 0.9974],
        [ 0.9974],
        [ 0.0072],
        [ 0.9974],
        [-0.0139]], grad_fn=<ThAddmmBackward>)


In [184]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, layer_sizes=[2, 20, 20, 1]):
        # num_obj_classes includes background class
        super(MLP, self).__init__()
        self.layers = []
        for i in range(len(layer_sizes)-1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.add_module('layer_{}'.format(i), self.layers[i])

    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = F.relu(self.layers[i](x))
        x = self.layers[-1](x)
        return x
    
model = MLP([2, 100, 25])
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.train()
for itr in range(1000):
    x = np.random.uniform(-1, 1, size=(100, 2))
    inds = (2*(x+1)).astype(np.uint8)
    targ = np.zeros((100, 5, 5), dtype=np.float32)
    for i in range(100):
        targ[i,inds[i][0],inds[i][1]] = 1.
    out = torch.nn.Softmax()(model(torch.Tensor(x)))
    out = torch.reshape(out, (-1, 5, 5))
    optimizer.zero_grad()
    loss = torch.mean(
        torch.abs(out - torch.Tensor(targ)),
    )
    loss.backward()
    optimizer.step()
print(loss)
# tests = []
# for x in range(-2, 3):
#     for y in range(-2, 3):
#         tests.append([x/2.,y/2.])
# tests = np.array(tests)
# print((2*(tests+1)).astype(np.uint8))
x = np.random.uniform(-1, 1, size=(100, 2))
inds = (2*(x+1)).astype(np.uint8)
targ = np.zeros((100, 5, 5), dtype=np.float32)
for i in range(100):
    targ[i,inds[i][0],inds[i][1]] = 1.
out = torch.nn.Softmax()(model(torch.Tensor(x)))
out = torch.reshape(out, (-1, 5, 5))
loss = torch.mean(
    torch.abs(out - torch.Tensor(targ)),
)
print(loss)
print(out)
print(targ)

/home/alvin/anaconda3/envs/rl/lib/python3.5/site-packages/ipykernel/__main__.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.0092, grad_fn=<MeanBackward1>)
tensor(0.0056, grad_fn=<MeanBackward1>)
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.8579, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.1421, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.8287, 0.1713, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0154, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9846, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        ...,

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,

/home/alvin/anaconda3/envs/rl/lib/python3.5/site-packages/ipykernel/__main__.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:
import h5py
import numpy as np
data = h5py.File('data/obj_balls.h5', 'r')

ims = np.zeros((5, 3, 64, 64), dtype=np.float32)

locs = np.where(data['training']['groups'][:1, :5][0] == 2)
ims[locs[0], np.zeros_like(locs[3]), locs[1], locs[2]] = 1.

locs = np.where(data['training']['groups'][:1, :5][0] == 1)
ims[locs[0], 2*np.ones_like(locs[3]), locs[1], locs[2]] = 1.

import cv2
cv2.imshow('im', (255*ims[0]).astype(np.uint8).transpose([1,2,0]))
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
 

In [12]:
from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, layer_sizes=[1, 100, 100, 100, 2]):
        # num_obj_classes includes background class
        super(MLP, self).__init__()
        self.layers = []
        for i in range(len(layer_sizes)-1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.add_module('layer_{}'.format(i), self.layers[i])

    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = F.elu(self.layers[i](x))
        x = self.layers[-1](x)
        return x
    
model = MLP([1, 100, 10])
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.train()
for itr in range(1000):
    inds = np.random.randint(0, 8, size=1000, dtype=np.uint8)
    x = np.zeros((1000, 10))
    y = np.zeros((1000, 10))
    x[np.arange(1000),inds] = 1
    x += np.random.normal(0, 0.1, size=(1000, 10))
    y[np.arange(1000),inds+2] = 1
    
    x_var = Variable(torch.Tensor(x), requires_grad=True)
#     reconstr = model(x_var)
    reconstr = torch.zeros((1000, 10))
    inds = (x_var >= 0.5).nonzero()
    reconstr = model(inds[:,1].type(torch.FloatTensor).reshape(-1, 1)) #for objects: reshape, and sum, and reshape again
#     shifts = Variable(model(torch.Tensor(np.expand_dims(inds, axis=1))), requires_grad=True)
# #     shifts = np.ones((1000, 2))*0.001
# #     shifts[:,0] = (inds + 2 - 4.5)/4.5
# #     shifts = torch.Tensor(shifts)
    
# #     m = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
# #     samples = m.sample(torch.Size([100]))[:,0]
# #     samples *= shifts[:,1]
# #     samples += shifts[:,0]
# #     samples = torch.clamp(samples, -4.5, 4.498)
# #     samples += 4.5
# #     reconstr = torch.zeros((100, 10))
# #     for i in range(100):
# #         reconstr[i].index_fill_(0, torch.floor(samples[i]).type(torch.LongTensor), samples[i]-torch.floor(samples[i]))
# #         reconstr[i].index_fill_(0, torch.ceil(samples[i]+0.001).type(torch.LongTensor), torch.ceil(samples[i]+0.001)-samples[i])
# #     print(reconstr[0], y[0])
#     reconstr = torch.zeros((1000, 10))
#     softmax = torch.nn.Softmax()
#     for i in range(1000):
#         row = torch.zeros(10)
#         for j in range(10):
#             row[j] = -((j-4.5)/4.5-shifts[i,0])**2 #/ shifts[i,1]
#         reconstr[i] = softmax(row)
# #         print(reconstr[i])
    optimizer.zero_grad()
    loss = torch.mean(
        (reconstr - torch.Tensor(y))**2,
    )
    loss.backward()
#     print(shifts.grad[0, 0])
#     print(x_var.grad)
    optimizer.step()
#     print(loss, (inds[0]+2-4.5)/4.5, shifts[0, 0])
    if itr % 100 == 0:
        print(loss, reconstr[0], y[0])
print(loss)

tensor(0.9666, grad_fn=<MeanBackward1>) tensor([-0.6943,  0.8889,  0.9955, -0.4030,  1.3577,  1.0361, -0.2839, -0.1639,
         0.7220,  0.3152], grad_fn=<SelectBackward>) [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
tensor(0.0566, grad_fn=<MeanBackward1>) tensor([ 0.0053,  0.0049, -0.0154,  0.2577,  0.3687,  0.2930,  0.1761,  0.0571,
        -0.0068, -0.1126], grad_fn=<SelectBackward>) [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


RuntimeError: The size of tensor a (1001) must match the size of tensor b (1000) at non-singleton dimension 0

In [58]:
from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class MLP(nn.Module):
    def __init__(self, layer_sizes=[1, 100, 100, 100, 2]):
        # num_obj_classes includes background class
        super(MLP, self).__init__()
        self.layers = []
        for i in range(len(layer_sizes)-1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.add_module('layer_{}'.format(i), self.layers[i])

    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = F.elu(self.layers[i](x))
        x = self.layers[-1](x)
        return x

mb_size = 1000
grid_size = 100
max_shift = 2
force_size = 1

force_model = MLP([1, 100, force_size])
render_model = MLP([force_size, 100, 2*max_shift+1])
optimizer = optim.Adam(list(force_model.parameters()) + list(render_model.parameters()), lr=0.01)
force_model.train()
render_model.train()

# ind_combs = np.dstack(np.meshgrid(np.arange(100), np.arange(100))).reshape(-1, 2)
# ind_combs = ind_combs[:, [1,0]]
# ind_combs = torch.Tensor(ind_combs)
diff_combs = np.arange(-grid_size+1, grid_size).reshape(-1, 1)
diff_combs = torch.Tensor(diff_combs)
diff_inds = np.dstack(np.meshgrid(np.arange(grid_size), np.arange(grid_size))).reshape(-1, 2)
diff_inds = diff_inds[:, 1] - diff_inds[:, 0] + grid_size - 1
diff_inds = torch.Tensor(diff_inds.reshape(-1, 1)).type(torch.LongTensor)
# res = []
# for i in range(x.shape[0]):
#     res.extend(np.concatenate((np.tile(x[i], (x.shape[0], 1)), x), axis=1))
# np.array(res)

for itr in range(1000):
    inds = np.random.randint(2*max_shift, grid_size-2*max_shift, size=mb_size, dtype=np.uint8)
#     inds = 4*np.ones(mb_size).astype(np.uint8)
    shifts = np.random.randint(-max_shift, max_shift+1, size=mb_size, dtype=np.int8)
    
    prev = np.zeros((mb_size, grid_size))
    cur = np.zeros((mb_size, grid_size))
    targ = np.zeros((mb_size, grid_size))
    
    prev[np.arange(mb_size),inds] = 1
    cur[np.arange(mb_size),inds+shifts] = 1
    targ[np.arange(mb_size),inds+2*shifts] = 1
    
    prev = Variable(torch.Tensor(prev), requires_grad=True)
    cur = Variable(torch.Tensor(cur), requires_grad=True)
    targ = Variable(torch.Tensor(targ), requires_grad=False)
    
    pred_forces = force_model(ind_combs.type(torch.FloatTensor))
#     pred_forces = ind_combs.type(torch.FloatTensor)
    forces = torch.zeros((mb_size, grid_size, force_size))
#     import time
#     start = time.time()
    for cur_loc in range(grid_size):
        forces[:, cur_loc] = torch.sum(
                cur[:, cur_loc].reshape(mb_size, 1, 1)
                * prev[:, :].reshape(mb_size, grid_size, 1)
                * pred_forces[diff_inds[cur_loc*grid_size : (cur_loc+1)*grid_size]].reshape(grid_size, force_size).repeat(mb_size, 1, 1),
                dim=1
            )
    pred_shifts = render_model(forces.reshape(-1, force_size)).reshape(mb_size, grid_size, 2*max_shift+1)
    render = torch.zeros((mb_size, grid_size+2*max_shift))
    for cur_loc in range(grid_size):
        render[:, cur_loc:cur_loc+2*max_shift+1] += cur[:, cur_loc].reshape(mb_size, 1) * pred_shifts[:, cur_loc]
#     end = time.time()
#     print('forward time', end-start)
#     start = time.time()
    render = render[:, max_shift:max_shift+grid_size]
    optimizer.zero_grad()
    loss = torch.mean(
        (render - targ)**2,
    )
    loss.backward()
    optimizer.step()
#     end = time.time()
#     print('backward time', end-start)
    if itr % 10 == 0:
        print(loss, shifts[0], render[0], targ[0])
#         print(pred_shifts[[97, 98, 99, 100, 101]])
print(loss)

tensor(0.0095, grad_fn=<MeanBackward1>) 2 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1998, -0.1047,  0.1026,
         0.0488,  0.2046,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

tensor(0.0051, grad_fn=<MeanBackward1>) -2 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.8371,  0.2776,  0.0787, -0.0245, -0.1722,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.000

tensor(0.0016, grad_fn=<MeanBackward1>) 0 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
        -0.0942,  0.1650,  0.7638,  0.3246, -0.1445,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

tensor(0.0003, grad_fn=<MeanBackward1>) 2 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000, -0.0273, -0.0388,  0.1428, -0.0797,  0.9189,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

KeyboardInterrupt: 